In [1]:
from graphai import node

@node(start=True)
def node_a(param_a: str):
    """Descriptive string for the node.
    """
    return {"param_b": "Hello, World!"}

@node(end=True)
def node_b(param_b: str):
    """Descriptive string for the node.
    """
    new_str = param_b*2
    return {"output": new_str}

In [2]:
node_a.get_signature()

"param_a: <class 'str'>"

In [3]:
node_a.invoke(input={"param_a": "l"})

{'param_b': 'Hello, World!'}

In [4]:
from graphai import Graph

graph = Graph()

In [5]:
graph.add_node(node_a)
graph.add_node(node_b)
graph.add_edge(source=node_a, destination=node_b)

In [6]:
graph.start_node, graph.end_nodes

(graphai.nodes.base._Node._node.<locals>.NodeClass,
 [graphai.nodes.base._Node._node.<locals>.NodeClass])

In [7]:
graph.execute(input={"param_a": "l"})

{'param_b': 'Hello, World!'}